# Configurações

## Importações

In [9]:
import pandas as pd

## Definição do Dataframe

In [10]:
import requests
import zipfile
import os
from pathlib import Path

# URL do dataset
url = "https://www.kaggle.com/api/v1/datasets/download/abdelrahman16/spotify-analysis-and-visualization"

# Criar diretório datasets se não existir
datasets_dir = Path("../datasets")
datasets_dir.mkdir(exist_ok=True)

# Caminho temporário para salvar o arquivo zip
temp_zip_path = Path("/tmp/spotify-analysis-and-visualization.zip")

print("Iniciando download do dataset...")

try:
    # Fazer o download
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Levanta exceção se houver erro HTTP
    
    # Salvar o arquivo zip temporariamente
    with open(temp_zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    print(f"Download concluído!")
    
    # Extrair o arquivo zip
    print("Extraindo arquivos...")
    with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
        # Listar arquivos no zip
        file_list = zip_ref.namelist()
        print(f"Arquivos encontrados no ZIP: {file_list}")
        
        # Extrair todos os arquivos
        zip_ref.extractall(datasets_dir)
    
    print("Extração concluída!")
    
    # Procurar por arquivos CSV e renomear o principal
    csv_files = list(datasets_dir.glob("*.csv"))
    if csv_files:
        # Assumindo que queremos o primeiro arquivo CSV encontrado
        original_csv = csv_files[0]
        new_csv_path = datasets_dir / "spotify_top_2000_songs.csv"
        
        # Renomear o arquivo se necessário
        if original_csv.name != "spotify_top_2000_songs.csv":
            original_csv.rename(new_csv_path)
            print(f"Arquivo renomeado de '{original_csv.name}' para 'spotify_top_2000_songs.csv'")
        else:
            print("Arquivo já possui o nome correto: 'spotify_top_2000_songs.csv'")
    
    # Remover o arquivo zip temporário
    temp_zip_path.unlink()
    print("Arquivo ZIP temporário removido.")
    
    # Listar arquivos finais na pasta datasets
    print("\nArquivos na pasta datasets:")
    for file in datasets_dir.iterdir():
        if file.is_file():
            print(f"- {file.name}")
        
except requests.exceptions.RequestException as e:
    print(f"Erro ao fazer download: {e}")
except zipfile.BadZipFile as e:
    print(f"Erro ao extrair arquivo zip: {e}")
except Exception as e:
    print(f"Erro inesperado: {e}")
finally:
    # Garantir que o arquivo temporário seja removido mesmo em caso de erro
    if temp_zip_path.exists():
        temp_zip_path.unlink()
        print("Arquivo ZIP temporário removido (cleanup).")

Iniciando download do dataset...
Download concluído!
Extraindo arquivos...
Arquivos encontrados no ZIP: ['songs_normalize.csv']
Extração concluída!
Arquivo já possui o nome correto: 'spotify_top_2000_songs.csv'
Arquivo ZIP temporário removido.

Arquivos na pasta datasets:
- spotify_top_2000_songs.csv
- songs_normalize.csv
Download concluído!
Extraindo arquivos...
Arquivos encontrados no ZIP: ['songs_normalize.csv']
Extração concluída!
Arquivo já possui o nome correto: 'spotify_top_2000_songs.csv'
Arquivo ZIP temporário removido.

Arquivos na pasta datasets:
- spotify_top_2000_songs.csv
- songs_normalize.csv


In [11]:
df = pd.read_csv('../datasets/spotify_top_2000_songs.csv')

# Conhecendo as Features

In [12]:
df.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


In [13]:
columns = df.columns

print('As features do dataset são:')
for(i, col) in enumerate(columns):
    print(f'{i+1}. {col}')

As features do dataset são:
1. artist
2. song
3. duration_ms
4. explicit
5. year
6. popularity
7. danceability
8. energy
9. key
10. loudness
11. mode
12. speechiness
13. acousticness
14. instrumentalness
15. liveness
16. valence
17. tempo
18. genre


# Limpeza do dados

## Análise de nulos

In [14]:
def verificar_nulos(dataframe):
    nulos = dataframe.isnull().sum()
    total = dataframe.shape[0]
    porcentagem = (nulos / total) * 100
    resultado = pd.DataFrame({'Nulos': nulos, 'Porcentagem (%)': porcentagem})
    return resultado

In [15]:
verificar_nulos(df)

,Nulos,Porcentagem (%)
artist,0,0.0
song,0,0.0
duration_ms,0,0.0
explicit,0,0.0
year,0,0.0
popularity,0,0.0
danceability,0,0.0
energy,0,0.0
key,0,0.0
loudness,0,0.0


## Análise descritiva

In [16]:
df.describe()

,duration_ms,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,228748.124500,2009.49400,59.872500,0.667438,0.720366,5.378000,-5.512434,0.553500,0.103568,0.128955,0.015226,0.181216,0.551690,120.122558
std,39136.569008,5.85996,21.335577,0.140416,0.152745,3.615059,1.933482,0.497254,0.096159,0.173346,0.087771,0.140669,0.220864,26.967112
min,113000.000000,1998.00000,0.000000,0.129000,0.054900,0.000000,-20.514000,0.000000,0.023200,0.000019,0.000000,0.021500,0.038100,60.019000
25%,203580.000000,2004.00000,56.000000,0.581000,0.622000,2.000000,-6.490250,0.000000,0.039600,0.014000,0.000000,0.088100,0.386750,98.985750
50%,223279.500000,2010.00000,65.500000,0.676000,0.736000,6.000000,-5.285000,1.000000,0.059850,0.055700,0.000000,0.124000,0.557500,120.021500
75%,248133.000000,2015.00000,73.000000,0.764000,0.839000,8.000000,-4.167750,1.000000,0.129000,0.176250,0.000068,0.241000,0.730000,134.265500
max,484146.000000,2020.00000,89.000000,0.975000,0.999000,11.000000,-0.276000,1.000000,0.576000,0.976000,0.985000,0.853000,0.973000,210.851000


# Gerar o dataframe

In [17]:
df.to_csv('../datasets/data_clean.csv', index=False, encoding='utf-8')

# Resultados

Aparentemente os dados desse dataset de 2000 músicas está tratado. Foi encontrado apenas uma inconsistência de algumas musicas possirem o gênero "set()", que muito provavelmente foi erro de input.